In [1]:
!pip install -q torch transformers accelerate peft trl bitsandbytes scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.4 MB/s eta 0:00:00


In [2]:
!pip install -q -U transformers peft accelerate bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 7.7 MB/s eta 0:00:00


In [3]:
!pip install bitsandbytes==0.41.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.48.2
    Uninstalling bitsandbytes-0.48.2:
      Successfully uninstalled bitsandbytes-0.48.2


In [4]:
!pip install -U "trl>=0.8.6" "transformers>=4.44" "peft>=0.12" "bitsandbytes>=0.43" "accelerate>=1.0"

  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.0
    Uninstalling bitsandbytes-0.41.0:
      Successfully uninstalled bitsandbytes-0.41.0


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install --upgrade accelerate peft trl

In [9]:
import torch
import bitsandbytes as bnb

print(torch.cuda.is_available())  # trebuie să fie True
!python -m bitsandbytes             # verifică backend-ul


True
=================== bitsandbytes v0.48.2 ===================
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
  libc: glibc-2.35
Python: 3.12.12
PyTorch: 2.8.0+cu126
  CUDA: 12.6
  HIP: N/A
  XPU: N/A
Related packages:
  accelerate: 1.11.0
  diffusers: 0.35.2
  numpy: 2.0.2
  pip: 24.1.2
  peft: 0.18.0
  safetensors: 0.6.2
  transformers: 4.57.1
  triton: 3.4.0
  trl: 0.25.1
PyTorch settings found: CUDA_VERSION=126, Highest Compute Capability: (7, 5).
Checking that the library is importable and CUDA is callable...
SUCCESS!


In [10]:
import os
from dataclasses import dataclass
from typing import Dict, List, Optional
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

TEMP_DATA_FOLDER = "/content/drive/MyDrive/DentalData/"


json_files = f"{TEMP_DATA_FOLDER}*.json"

txt_files = f"{TEMP_DATA_FOLDER}*.txt"

FILE_NAME = [json_files, txt_files]

raw_ds = load_dataset("json", data_files=FILE_NAME, split="train")
print(f"Loaded {len(raw_ds)} examples.")

MODEL_ID = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"
# Configurația QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    no_cuda=True
)

# Configurația LoRA (PEFT)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:

    tokenizer.pad_token = tokenizer.eos_token

def map_to_text(example):
    msgs = example.get("messages", [])
    norm = []
    for m in msgs:
        role = m.get("role", "user")
        if role not in ("system", "user", "assistant"):
            role = "user"
        content = (m.get("content") or "").strip()
        if not content:
            continue
        norm.append({"role": role, "content": content})
    text = tokenizer.apply_chat_template(
        norm,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

ds = raw_ds.map(map_to_text, remove_columns=raw_ds.column_names)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

model = get_peft_model(model, lora_config)
model.gradient_checkpointing_enable()

print(f"Numar de parametri antrenabili: {model.print_trainable_parameters()}")
ASSISTANT_TRIGGER = "<|start_header_id|>assistant<|end_header_id|>"

from dataclasses import dataclass
from typing import Dict, List, Optional
import torch

@dataclass
class TokenizedCompletionOnlyCollator:
    tokenizer: AutoTokenizer
    response_template: str
    max_length: int = 1024
    pad_to_multiple_of: Optional[int] = 8

    def _pad(self, sequences: List[List[int]], pad_id: int) -> torch.Tensor:
        max_len = max(len(s) for s in sequences)
        if self.pad_to_multiple_of:
            rem = max_len % self.pad_to_multiple_of
            if rem != 0:
                max_len += (self.pad_to_multiple_of - rem)
        out = []
        for s in sequences:
            out.append(s + [pad_id] * (max_len - len(s)))
        return torch.tensor(out, dtype=torch.long)

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        if "input_ids" in features[0]:
            input_ids_list = [f["input_ids"] for f in features]
            attn_list = [f.get("attention_mask", [1]*len(f["input_ids"])) for f in features]

            input_ids = self._pad(input_ids_list, self.tokenizer.pad_token_id)
            attention_mask = self._pad(attn_list, 0)

        else:
            texts = [f["text"] for f in features]
            enc = self.tokenizer(
                texts,
                max_length=self.max_length,
                truncation=True,
                padding=True,
                return_tensors="pt"
            )
            input_ids = enc["input_ids"]
            attention_mask = enc["attention_mask"]

        labels = input_ids.clone()


        full_assistant_template = ASSISTANT_TRIGGER + "\n"

        templ_ids = self.tokenizer(full_assistant_template, add_special_tokens=False).input_ids
        tlen = len(templ_ids)

        for i in range(input_ids.size(0)):
            row = input_ids[i].tolist()
            start_idx = -1
            for j in range(0, len(row) - tlen + 1):
                if row[j:j+tlen] == templ_ids:
                    start_idx = j + tlen
                    break
            if start_idx > 0:
                labels[i, :start_idx] = -100

        labels[attention_mask == 0] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

collator = TokenizedCompletionOnlyCollator(
    tokenizer=tokenizer,
    response_template=ASSISTANT_TRIGGER,
    max_length=1024
)


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    logging_steps=5,
    save_steps=100,
    save_total_limit=2,
    optim="paged_adamw_32bit",
    fp16=True,
    bf16=False,
    report_to="none",
    max_steps=300,
)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    peft_config=lora_config,
    data_collator=collator,
)

try:
    trainer.train()
finally:
    trainer.model.save_pretrained("./final")
    tokenizer.save_pretrained("./final")
    print("Model salvat temporar în ./final")

Generating train split: 0 examples [00:00, ? examples/s]

Loaded 367 examples.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/367 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/367 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss
5,2.116400
10,1.385500
15,1.107400
20,0.857000
25,0.583300
30,0.507900
35,0.441500
40,0.329600
45,0.211500
50,0.222700


Model salvat temporar în ./final


In [13]:
!zip -r final.zip /content/final
!zip -r final.zip /content/results

  adding: content/final/ (stored 0%)
  adding: content/final/adapter_config.json (deflated 58%)
  adding: content/final/adapter_model.safetensors (deflated 8%)
  adding: content/final/merges.txt (deflated 57%)
  adding: content/final/README.md (deflated 65%)
  adding: content/final/added_tokens.json (deflated 67%)
  adding: content/final/vocab.json (deflated 61%)
  adding: content/final/tokenizer_config.json (deflated 89%)
  adding: content/final/chat_template.jinja (deflated 71%)
  adding: content/final/tokenizer.json (deflated 81%)
  adding: content/final/special_tokens_map.json (deflated 69%)
  adding: content/results/ (stored 0%)
  adding: content/results/checkpoint-300/ (stored 0%)
  adding: content/results/checkpoint-300/adapter_config.json (deflated 58%)
  adding: content/results/checkpoint-300/adapter_model.safetensors (deflated 8%)
  adding: content/results/checkpoint-300/merges.txt (deflated 57%)
  adding: content/results/checkpoint-300/training_args.bin (deflated 53%)
  addi

In [ ]:
import torch, os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

BASE_ID = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"
ADAPTER_DIR = "/content/final/"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base = AutoModelForCausalLM.from_pretrained(BASE_ID, quantization_config=bnb, device_map="auto")
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()

def build_patient_prompt(pathology, mf):
    return f"""You are the PATIENT (the assistant). DO NOT reveal your diagnosis.
Pathology: {pathology}

INSTRUCTIONS:
1) Answer naturally, like a real human patient — short, realistic sentences.
2) Stay consistent with the symptoms below.
3) If the doctor tries to guess or ask for the diagnosis, don't confirm it directly. Describe only your symptoms.
4) If asked about something not listed, answer plausibly without contradicting the rest.

SYMPTOMS / CONTEXT:
- Main complaint: {mf.get('chief_complaint','')}
- Pain: {mf.get('pain','')}
- Location: {mf.get('location','')}
- Onset/duration: {mf.get('duration','')}
- Aggravating factors: {mf.get('aggravating','')}
- Relieving factors: {mf.get('relieving','')}
- Appearance: {mf.get('appearance','')}
- History: {mf.get('history','')}
- Other info: {mf.get('extra','')}
"""

manifest = {
    "chief_complaint": "My tooth has turned gray, but it doesn't hurt anymore.",
    "pain": "No pain to cold, heat, or spontaneously. Only slight tenderness when biting or tapping.",
    "location": "Lower right molar.",
    "duration": "Severe pain 2–4 weeks ago, which has completely stopped since then.",
    "appearance": "Tooth looks grayish.",
    "history": "Past episodes of strong spontaneous or night pain.",
}
SYSTEM_PROMPT = build_patient_prompt("Pulpal Necrosis", manifest)

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Good afternoon. What seems to be the problem today?"}
]

def generate_reply(messages, max_new_tokens=150, temperature=0.3, top_p=0.9):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.inference_mode():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )


    gen_tokens = output[0, inputs["input_ids"].shape[-1]:]
    reply = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
    return reply

print(" Doctor–Patient Chat started (English). Type /reset or /exit to quit.\n")

assistant_reply = generate_reply(messages)
print(f"Patient: {assistant_reply}\n")
messages.append({"role": "assistant", "content": assistant_reply})

while True:
    user = input("Doctor: ").strip()
    if user.lower() == "/exit":
        break
    if user.lower() == "/reset":
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]
        print("Context reset.\n")
        continue

    messages.append({"role": "user", "content": user})
    assistant_reply = generate_reply(messages)
    print(f"\nPatient: {assistant_reply}\n")
    messages.append({"role": "assistant", "content": assistant_reply})


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Doctor–Patient Chat started (English). Type /reset or /exit to quit.

Patient: It's not that it hurts now, but I feel a strange taste sometimes.

Doctor: When does it hurt?

Patient: I haven't had any pain at all for a while.

Doctor: What kind of taste do you feel?

Patient: Something with a salty flavor came out of it once.

